In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2
import random
import plotly.express as px
from Levenshtein import distance

from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.image import resize

# Load .csv file

In [ ]:
# train labels
train_labels = pd.read_csv("bms-molecular-translation/train_labels.csv")
train_labels['InChI'] = train_labels['InChI'].apply(lambda x: x.replace('InChI=', ''))
train_labels = train_labels.set_index("image_id")
print(f"Size training set: {len(train_labels)}")

In [ ]:
# Text processing
text = ''.join(train_labels['InChI'].values)

# Vocab
vocab = [' '] + sorted(set(text))
vocab_size = len(vocab)

# Mapping
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Max length
max_len = max([len(x) for x in train_labels['InChI']])

# Load image data, create target and train

In [ ]:
# Random seed
random_state=0

# Parameters
epochs = 10
batch_size = 32
lr=1e-3
name=f'gsk'
new_shape=[128, 128]

# Optimizer
optimizer = Adam(learning_rate=lr)

In [ ]:
# Model 
model = get_model(max_len, vocab)
model.compile(optimizer=optimizer, loss=loss_function)
model.summary()

In [ ]:
# Images data 
folders = '0123456789abcdef'
dataset = 'train'

i = folders[0]
j = folders[0]
k = folders[0]

# for i in folders[0:1]:
#     for j in folders[0:1]:
#         for k in folders[1:2]:

print(f"### {i} - {j} - {k} ###")

path = f'bms-molecular-translation/{dataset}/{i}/{j}/{k}/'

for e in range(1):
    
    print(f"# Epoch: {e}")
    
    score_list = []

    # Files
    list_names = os.listdir(path)
    list_paths = [path for _ in list_names]

    # Image data
    ImageSet = ImageSetObject(list_names, list_paths)
    ImageSet.load_set(new_shape)
    data = ImageSet.X

    # Text targets
    list_id = [x.split('.')[0] for x in ImageSet.list_names]
    targets = train_labels.loc[list_id, 'InChI'].values
    targets = [[char2idx[x] for x in target] for target in targets]
    targets = pad_sequences(targets, padding='post', maxlen=max_len)

    # Train
    history = model.fit(data, targets, epochs=100, batch_size=batch_size, verbose=1)
    model.save_weights(f'outputs/{name}.h5')

    # Score 
    y_true=[''.join([idx2char[int(y)] for y in yy]) for yy in targets]
    y_predict=get_text_from_predict(model, data, idx2char)
    score_list.append(score(y_true, y_predict))
    print(f"\t> Score: {np.mean(score_list)}")

In [ ]:
y_true=[''.join([idx2char[int(y)] for y in yy]) for yy in targets]
y_predict=get_text_from_predict(model, data, idx2char)
score(y_true, y_predict)

In [ ]:
y_true[1]

In [ ]:
y_predict[1]

In [ ]:
char2idx

# Validation

In [ ]:
# Images data 
dataset = 'train'

i = folders[5]
j = folders[5]
k = folders[5]

print(f"### {i} - {j} - {k} ###")

path = f'bms-molecular-translation/{dataset}/{i}/{j}/{k}/'

In [ ]:
# Files
list_names = os.listdir(path)
list_paths = [path for _ in list_names]

# Image data
ImageSet = ImageSetObject(list_names, list_paths)
ImageSet.load_set(new_shape)
data_validation = ImageSet.X

# Text targets
list_id = [x.split('.')[0] for x in ImageSet.list_names]
targets = train_labels.loc[list_id, 'InChI'].values
targets = [[char2idx[x] for x in target] for target in targets]
targets = pad_sequences(targets, padding='post', maxlen=max_len)

In [ ]:
# Predict
y_val_true=[''.join([idx2char[int(y)] for y in yy]) for yy in targets_validation]
y_val_predict=get_text_from_predict(model, data_validation, idx2char)
score(y_val_true, y_val_predict)

In [ ]:
y_val_true[1]

In [ ]:
y_val_predict[1]

# Submission

In [ ]:
# # Create tf model
# model = get_model(max_len, vocab)

# # load weights
# try:
#     model.load_weights(f'outputs/{name}.h5')
#     print("Loaded model from disk")
# except:
#     print("No initial model")

# model.compile(optimizer=optimizer, loss=loss_function)

In [ ]:
# sample_submission
sample_submission = pd.read_csv("bms-molecular-translation/sample_submission.csv")
sample_submission = sample_submission.set_index('image_id')

In [ ]:
# Images data 
dataset = 'test'

i = folders[5]
j = folders[5]
k = folders[5]

print(f"### {i} - {j} - {k} ###")

path = f'bms-molecular-translation/{dataset}/{i}/{j}/{k}/'

In [ ]:
# Files
list_names = os.listdir(path)
list_paths = [path for _ in list_names]

# Image data
ImageSet = ImageSetObject(list_names, list_paths)
ImageSet.load_set(new_shape)
data_test = ImageSet.X

# Text targets
list_id = [x.split('.')[0] for x in ImageSet.list_names]

In [ ]:
# Predict
y_test_predict=get_text_from_predict(model, data_validation, idx2char)
y_test_predict=['InChI='+x for x in y_test_predict]

In [ ]:
df_y_test_predict = pd.DataFrame([list_id, y_test_predict], index = ['image_id','InChI']).transpose().set_index('image_id')

In [ ]:
sample_submission.loc[df_y_test_predict.index, 'InChI'] = df_y_test_predict['InChI']

In [ ]:
# Export
sample_submission = sample_submission.reset_index()

In [ ]:
sample_submission.to_csv('outputs/submission.csv', index=False)